# Experimentos XGBoost x SVM; TFIDF x Embeddings

## Fontes primárias:
- Tutorial Using XGBoost in Python: https://www.datacamp.com/community/tutorials/xgboost-in-python
- Documentação XGBoost Python API: https://xgboost.readthedocs.io/en/latest/python/python_api.html
- XGBoost Python Package Introduction: https://xgboost-clone.readthedocs.io/en/latest/python/python_intro.html
- Hyperparameter tuning in XGBoost: https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f
- Linear SVC - Scikit Learn: https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn-svm-linearsvc
- precision_recall_fscore_support - Scikit Learn: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
- TfidfVectorizer - Scikit Learn: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- Embeddings NILC: http://www.nilc.icmc.usp.br/embeddings
- Inferring the source of official texts: can SVM beat ULMFiT?: https://cic.unb.br/~teodecampos/KnEDLe/propor2020/

## O que este caderno se propôs a fazer
Foram realizados testes com diferentes classificadores e representações textuais em um mesmo conjunto de dados, referente ao paper Inferring the source of official texts: can SVM beat ULMFiT?, disponível em https://cic.unb.br/~teodecampos/KnEDLe/propor2020/. As combinações estão listadas abaixo.

- TFIDF + SVM
- TFIDF + XGBoost
- CBOW FastText 300D + SVM
- SKIP-GRAM FastText 300D + SVM
- SKIP-GRAM Word2Vec 300D + SVM
- Glove 300D + SVM
- CBOW FastText 300D + XGBoost
- SKIP-GRAM FastText 300D + XGBoost
- SKIP-GRAM Word2Vec 300D + XGBoost
- Glove 300D + XGBoost

### Import do conjunto de dados:
 - Arquivo train.csv.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [2]:
path_to_text = '/home/mstauffer/Documentos/UnB/9º Semestre/KnEDle/sprints/5_27_maio-03_junho/luz_de_araujo_etal_propor2020/data/clean/train.csv'
data_peter = pd.read_csv(path_to_text, encoding='utf8')#[['v1', 'v2']]
# Creating the feature set and label set
textPT = data_peter['text']
labelPT = data_peter['label']
print(data_peter[10:14])

                                                label  \
10  SECRETARIA DE ESTADO DE DESENVOLVIMENTO URBANO...   
11  SECRETARIA DE ESTADO DE FAZENDA, PLANEJAMENTO,...   
12                      SECRETARIA DE ESTADO DE SAÚDE   
13          SECRETARIA DE ESTADO DE SEGURANÇA PÚBLICA   

                                                 text  is_valid  
10  O Termo de Recebimento Definitivo declarará fo...     False  
11  O DISTRITO FEDERAL, por intermédio da Diretori...     False  
12  O SECRETÁRIO DE ESTADO DE SAÚDE DO DISTRITO FE...     False  
13  O DIRETOR-GERAL DO DEPARTAMENTO DE TRÂNSITO DO...     False  


### Pré-processamento para baseline
- Processamos os dados de texto em representação TFIDF para usar como baseline

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = list(textPT)
X_tfidf = TfidfVectorizer(max_features=6000) 
X_tfidf.fit(corpus)
X_tfidf_features = X_tfidf.transform(corpus)
print(X_tfidf_features.shape)

(717, 6000)


- Pré-processamento do conjunto de labels

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support
# Converting the labels from strings to binary
le = LabelEncoder()
le.fit(labelPT)
y_labelPT = le.transform(labelPT)
print(y_labelPT)
print(y_labelPT.shape)

[15  6 15 11  0 13 10  2 14 18  8 11 14 15  4 18 15 15 10 10 10 12 11 11
 15 15 11  6 14 15 11  0 16 11  1 18 15  8 13 10  6 15  6 15 16 10  8 12
 11 15 15 13 16 15  3  5  3  6 11 15  0  2 15 15 11 15 16 14  5 15 14  2
 18 14 11 14  3 12 16  6 16 15 13 12  2 15  6 13  0 13  0 12 18 14  9 16
 14 14 15 11 15 15 13 15 13 15 14 13 18 11 14  7 15 18 16  6 10 16  0 11
  6 16 10 17  0  8 15 15  6 11 10  8 16 11 10 15 10 14 16  0 15 15 14  3
 11 14 11  0  3 15  7  7 14 12 18  2 13 13 18 18 10  0  0 15 14 16 18  5
 14 11 16  8 11 15 15 16 18 11 14 11 16  3  6 10 15  0  0 14 10  6 15 15
 10 15  0  0 14 13 14 15 15  2 16 13 15 10 12 16 11 11 13 11 14 15 15 10
  4 14 14 14 15 14 13 14  0  1  0  9 10  2 10 16 13  3 11 15  6  0 15  3
 13  0 10 11  0 15 15 11 15  3 15 15 15  3 18 15 16  6  1 16 16 14 15 14
 13 14 18  6 15  3  3 14 13  0 16  0 15  5 13 15 15 16 14 18 15  0 13 14
 15 13 15  0  0 11 14 15 13 15  0  0 15 15  8 15 15 11 14  6 15 14 14  0
 16  2  6  2 17 13  7  2 15 16  3 14 13 16 15  2  0

### Split em treino e teste - baseline
- OBS: Os labels também serão utilizados com outras representações.

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf_features, y_labelPT, test_size=0.2, random_state=123)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

  (0, 5865)	0.12310278357902607
  (0, 5844)	0.07214349812284936
  (0, 5835)	0.0868076648446096
  (0, 5754)	0.041587122710311904
  (0, 5743)	0.10162001575332077
  (0, 5668)	0.028382729835351277
  (0, 5420)	0.2645848762644777
  (0, 5401)	0.05307699402489731
  (0, 5400)	0.04830196249802336
  (0, 5385)	0.07645466551500592
  (0, 5356)	0.09638715650117681
  (0, 5349)	0.05393150042329729
  (0, 5331)	0.0282035262524768
  (0, 5270)	0.2223558217390038
  (0, 5251)	0.07931376976638826
  (0, 5130)	0.05583318328674835
  (0, 5018)	0.09662967176577104
  (0, 4963)	0.052275930237077724
  (0, 4957)	0.09193180845870408
  (0, 4956)	0.06077461776690305
  (0, 4929)	0.2379413092991648
  (0, 4863)	0.1494332956490792
  (0, 4832)	0.05635622926945937
  (0, 4831)	0.05128648248887804
  (0, 4790)	0.06575128557963889
  :	:
  (572, 855)	0.10020939169551783
  (572, 754)	0.09235404367340458
  (572, 615)	0.09712011448381817
  (572, 575)	0.13776221649461948
  (572, 537)	0.08245792432452834
  (572, 517)	0.07229381086033336

## Baseline SVM + TFIDF

In [ ]:
svm_modelPT = LinearSVC(max_iter=1000)
svm_modelPT.fit(X_train, y_train)
svm_predictionPT = svm_modelPT.predict(X_test)

## XGBoost

### Instanciação de objeto XGBClassifier
Instanciação e fit do objeto com X e y de treino. Call do método predict e store em variável para análise de perfomance.

Refs: 
- https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier
- https://info.cambridgespark.com/latest/getting-started-with-xgboost

In [35]:
xg_cls = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, objective='multi:softprob', colsample_bytree = 1, gamma=0, learning_rate = 0.1, max_delta_step=0,
                max_depth = 3, min_child_weight=1, missing=None, alpha = 10, n_estimators = 100, nthread=-1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1)

In [36]:
xg_cls.fit(X_train, y_train)

[17:35:26] WARNING: /workspace/src/learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
preds = xg_cls.predict(X_test)

In [76]:
print(preds)

[14 14 14  0  6 16  6 14  0 15 10 15  0  0 16 15 10  6 15  8 15 13 14  0
 15 11 15 12 18 15 15 11  0 10 11  3 15 16 15 12 15 13 11  6 13  6 16 15
  3 15 15 14  2  2 14 14 13 13 10 14 16 11  6  6 13  9 15 14 16  6  3 10
 16 11 12 11 13 13 15 18 10 18  6  3  6 15 14 11 11 14  2 15 16 11 18  5
  5 14 15 10 13 11 13 14 18  1  0 13 11 13 16 13  0 10 13 14 13 11  3  7
 15 18 18  0 10  8 15 16 14 11 15 15 15 11  0 11 11  0 15 15 15  3 14 16]


## Modelos baseados em Embeddings

### Import de embeddings

Foram conduzidos testes com diferentes conjuntos de vetores pré-treinados em língua portuguesa. Os arquivos usados estão disponíveis em http://www.nilc.icmc.usp.br/embeddings. A saber, são eles:
- FastText CBOW 300 dimensões
- FastText SKIP-GRAM 300 dimensões
- Word2Vec SKIP-GRAM 300 dimensões
- Glove 300 dimensões

A fim de preservar memória, apenas as 100000 palavras mais frequentes foram usadas. Além disso, as duas células abaixo foram executadas sobrescrevendo as variáveis envolvidas a cada mudança de conjunto de embeddings. Apenas uma linha era modificada (path_emb = 'modelo'). Carregadas as informações do modelo, os testes seguiam.

In [107]:
# Loading the data file from local download
path_emb = 'glove_s300.txt'
dictionary = open(path_emb, 'r', encoding='utf-8',
                  newline='\n', errors='ignore')
embedsPT = {}
for line in dictionary:
    tokens = line.rstrip().split(' ')
    embedsPT[tokens[0]] = [float(x) for x in tokens[1:]]
    
    if len(embedsPT) == 100000:
        break
print(embedsPT['carro'])

[-0.457517, -0.08992, 0.344763, 0.041031, 0.072887, -0.974737, 0.160377, -0.719449, -0.217797, 0.267011, -0.652553, 0.331003, 0.089468, -0.116183, 0.74115, -0.242831, -0.525771, 0.168315, -0.480077, 0.419809, 0.099425, -0.494544, 0.697174, 0.14046, -0.31801, -0.249461, -0.979818, -0.400552, -0.114748, 0.189945, 0.367923, -0.060311, -0.320324, -0.68412, -0.429739, -0.151582, 0.007587, 0.710586, -0.251371, 0.561936, -0.402018, -0.443148, -0.1371, -0.400934, -0.470808, -0.318996, -0.451779, -0.220812, 0.893261, 0.561035, -0.215192, -0.376958, 0.099948, 0.247093, -0.459781, 0.470335, 0.68523, 0.58447, -0.273652, -0.506633, -0.006587, 0.459712, 0.005831, 0.173205, -0.295872, -0.220857, 0.161053, -0.484545, 0.114365, 0.381987, -0.29784, -0.511537, 0.324228, 0.200679, 0.109674, -0.0143, -0.144205, -0.395351, 0.105982, 0.283204, 0.04553, -0.007537, 0.983664, -0.276508, 0.625008, -0.170508, 0.34148, -0.216058, -0.120424, -0.014769, 0.765968, -0.129805, 0.596335, 0.431591, -0.114619, 0.118298, 0

In [108]:
from keras.preprocessing.text import text_to_word_sequence
array_length = 20 * 300
embedding_featuresPT = pd.DataFrame()
i = 0
for document in textPT:
    i = i + 1
    # Saving the first 20 words of the document as a sequence
    words = text_to_word_sequence(document)[0:20] 
    
    # Retrieving the vector representation of each word and 
    # appending it to the feature vector 
    feature_vector = []
    for word in words:
        try:
            feature_vector = np.append(feature_vector, 
                                       np.array(embedsPT[word]))
        except KeyError:
            # In the event that a word is not included in our 
            # dictionary skip that word
            pass
    # If the text has less then 20 words, fill remaining vector with
    # zeros
    zeroes_to_add = array_length - len(feature_vector)
    print('Iteração %d' %i)
    print('Zeros_to_add: %d' %zeroes_to_add)
    feature_vector = np.append(feature_vector, 
                               np.zeros(zeroes_to_add)
                               ).reshape((1,-1))
    
    # Append the document feature vector to the feature table
    embedding_featuresPT = embedding_featuresPT.append( 
                                     pd.DataFrame(feature_vector))
print(embedding_featuresPT.shape)

Iteração 1
Zeros_to_add: 300
Iteração 2
Zeros_to_add: 2100
Iteração 3
Zeros_to_add: 300
Iteração 4
Zeros_to_add: 600
Iteração 5
Zeros_to_add: 300
Iteração 6
Zeros_to_add: 1500
Iteração 7
Zeros_to_add: 300
Iteração 8
Zeros_to_add: 0
Iteração 9
Zeros_to_add: 300
Iteração 10
Zeros_to_add: 0
Iteração 11
Zeros_to_add: 600
Iteração 12
Zeros_to_add: 0
Iteração 13
Zeros_to_add: 300
Iteração 14
Zeros_to_add: 300
Iteração 15
Zeros_to_add: 1200
Iteração 16
Zeros_to_add: 0
Iteração 17
Zeros_to_add: 1500
Iteração 18
Zeros_to_add: 900
Iteração 19
Zeros_to_add: 0
Iteração 20
Zeros_to_add: 0
Iteração 21
Zeros_to_add: 0
Iteração 22
Zeros_to_add: 0
Iteração 23
Zeros_to_add: 0
Iteração 24
Zeros_to_add: 0
Iteração 25
Zeros_to_add: 1200
Iteração 26
Zeros_to_add: 2700
Iteração 27
Zeros_to_add: 900
Iteração 28
Zeros_to_add: 2700
Iteração 29
Zeros_to_add: 300
Iteração 30
Zeros_to_add: 300
Iteração 31
Zeros_to_add: 900
Iteração 32
Zeros_to_add: 0
Iteração 33
Zeros_to_add: 0
Iteração 34
Zeros_to_add: 0
Iteração

Iteração 277
Zeros_to_add: 1200
Iteração 278
Zeros_to_add: 0
Iteração 279
Zeros_to_add: 1500
Iteração 280
Zeros_to_add: 1200
Iteração 281
Zeros_to_add: 300
Iteração 282
Zeros_to_add: 300
Iteração 283
Zeros_to_add: 300
Iteração 284
Zeros_to_add: 0
Iteração 285
Zeros_to_add: 300
Iteração 286
Zeros_to_add: 300
Iteração 287
Zeros_to_add: 1200
Iteração 288
Zeros_to_add: 0
Iteração 289
Zeros_to_add: 300
Iteração 290
Zeros_to_add: 0
Iteração 291
Zeros_to_add: 1800
Iteração 292
Zeros_to_add: 0
Iteração 293
Zeros_to_add: 300
Iteração 294
Zeros_to_add: 600
Iteração 295
Zeros_to_add: 0
Iteração 296
Zeros_to_add: 1200
Iteração 297
Zeros_to_add: 900
Iteração 298
Zeros_to_add: 1200
Iteração 299
Zeros_to_add: 600
Iteração 300
Zeros_to_add: 0
Iteração 301
Zeros_to_add: 300
Iteração 302
Zeros_to_add: 1500
Iteração 303
Zeros_to_add: 3000
Iteração 304
Zeros_to_add: 300
Iteração 305
Zeros_to_add: 1200
Iteração 306
Zeros_to_add: 600
Iteração 307
Zeros_to_add: 0
Iteração 308
Zeros_to_add: 2700
Iteração 309


Iteração 561
Zeros_to_add: 0
Iteração 562
Zeros_to_add: 0
Iteração 563
Zeros_to_add: 300
Iteração 564
Zeros_to_add: 0
Iteração 565
Zeros_to_add: 300
Iteração 566
Zeros_to_add: 0
Iteração 567
Zeros_to_add: 0
Iteração 568
Zeros_to_add: 0
Iteração 569
Zeros_to_add: 0
Iteração 570
Zeros_to_add: 900
Iteração 571
Zeros_to_add: 300
Iteração 572
Zeros_to_add: 0
Iteração 573
Zeros_to_add: 0
Iteração 574
Zeros_to_add: 1200
Iteração 575
Zeros_to_add: 0
Iteração 576
Zeros_to_add: 600
Iteração 577
Zeros_to_add: 2700
Iteração 578
Zeros_to_add: 2700
Iteração 579
Zeros_to_add: 900
Iteração 580
Zeros_to_add: 900
Iteração 581
Zeros_to_add: 0
Iteração 582
Zeros_to_add: 0
Iteração 583
Zeros_to_add: 1500
Iteração 584
Zeros_to_add: 0
Iteração 585
Zeros_to_add: 900
Iteração 586
Zeros_to_add: 300
Iteração 587
Zeros_to_add: 0
Iteração 588
Zeros_to_add: 1800
Iteração 589
Zeros_to_add: 2700
Iteração 590
Zeros_to_add: 2700
Iteração 591
Zeros_to_add: 0
Iteração 592
Zeros_to_add: 300
Iteração 593
Zeros_to_add: 0
It

### Split em treino e teste dos conjutos carregados de embeddings
Novamente, assim como ao carregar o modelo, aqui as variáveis foram sobrescritas conforme atualização do diferentes conjuntos de vetores pré-treinados.

In [109]:
from sklearn.model_selection import train_test_split

emb_train, emb_test = train_test_split(embedding_featuresPT, test_size=0.2, random_state=123)

print(emb_train.shape, emb_test.shape)
print(y_test.shape)

(573, 6000) (144, 6000)
(144,)


### XGBoost + Embeddings

In [87]:
xg_cls_emb = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, objective='multi:softprob', colsample_bytree = 1, gamma=0, learning_rate = 0.1, max_delta_step=0,
                max_depth = 3, min_child_weight=1, missing=None, alpha = 10, n_estimators = 100, nthread=-1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1)

Uma vez instanciado o objeto XGBClassifier, o reutilizamos a cada novo modelo de embeddings carregado, rodando o método fit com as embeddings de treino do modelo vigente e guardando em uma variável única o retorno do método predict, usado nas embeddings de teste do método vigente.

- XGBoost + Glove 300D

In [110]:
xg_cls_emb.fit(emb_train, y_train)

[20:59:23] WARNING: /workspace/src/learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [111]:
preds_glove_xgb = xg_cls_emb.predict(emb_test)

- XGBoost + Word2Vec SKIP-GRAM 300D

In [101]:
xg_cls_emb.fit(emb_train, y_train)

[20:39:15] WARNING: /workspace/src/learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [102]:
preds_w2vskip300_xgb = xg_cls_emb.predict(emb_test)

- XGBoost + FastText CBOW 300D

In [95]:
xg_cls_emb.fit(emb_train, y_train)

[20:30:58] WARNING: /workspace/src/learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [96]:
preds_cbow300ft_xgb = xg_cls_emb.predict(emb_test)

- XGBoost + FastText SKIP-GRAM 300D

In [88]:
xg_cls_emb.fit(emb_train, y_train)

[20:18:35] WARNING: /workspace/src/learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [89]:
preds_skip300ft_xgb = xg_cls_emb.predict(emb_test)

### SVM + Embeddings

- SVM + Glove 300D

In [115]:
svm_model_emb_glove300 = LinearSVC(max_iter=40000)
svm_model_emb_glove300.fit(emb_train, y_train)
svm_pred_emb_glove300 = svm_model_emb_glove300.predict(emb_test)

- SVM + Word2Vec SKIP-GRAM 300D

In [105]:
svm_model_emb_w2vskip300 = LinearSVC(max_iter=10000)
svm_model_emb_w2vskip300.fit(emb_train, y_train)
svm_pred_emb_w2vskip300 = svm_model_emb_w2vskip300.predict(emb_test)

- SVM + FastText CBOW 300D

In [55]:
svm_model_emb_cbow300ft = LinearSVC(max_iter = 40000)
svm_model_emb_cbow300ft.fit(emb_train, y_train)
svm_pred_emb_cbow300ft = svm_model_emb_cbow300ft.predict(emb_test)

/home/mstauffer/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


- SVM + FastText SKIP-GRAM 300D

In [44]:
svm_model_emb = LinearSVC(max_iter=10000)
svm_model_emb.fit(emb_train, y_train)
svm_prediction_emb = svm_model_emb.predict(emb_test)

## Tabela de resultados

In [116]:
resultsPT = pd.DataFrame(columns = ['Precision', 'Recall', 'F1 score', 'support']
          )
resultsPT.loc['TFIDF + SVM'] = precision_recall_fscore_support(
          y_test, 
          #labelPT[train_cutoff + 1 : len(text)], 
          svm_predictionPT, 
          average = 'weighted'
          )
resultsPT.loc['TFIDF + XGBoost'] = precision_recall_fscore_support(
          y_test, 
          preds, 
          average = 'weighted'
          )
resultsPT.loc['Emb FT Skip 300D + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_prediction_emb, 
          average = 'weighted'
          )
resultsPT.loc['Emb FT CBOW 300D + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_pred_emb_cbow300ft, 
          average = 'weighted'
          )
resultsPT.loc['Emb W2V Skip 300D + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_pred_emb_w2vskip300, 
          average = 'weighted'
          )
resultsPT.loc['Emb Glove 300D + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_pred_emb_glove300, 
          average = 'weighted'
          )
resultsPT.loc['Emb FT Skip 300D + XGBoost'] = precision_recall_fscore_support(
          y_test, 
          preds_skip300ft_xgb, 
          average = 'weighted'
          )
resultsPT.loc['Emb FT CBOW 300D + XGBoost'] = precision_recall_fscore_support(
          y_test, 
          preds_cbow300ft_xgb, 
          average = 'weighted'
          )
resultsPT.loc['Emb W2V Skip 300D + XGBoost'] = precision_recall_fscore_support(
          y_test, 
          preds_w2vskip300_xgb, 
          average = 'weighted'
          )
resultsPT.loc['Emb Glove 300D + XGBoost'] = precision_recall_fscore_support(
          y_test, 
          preds_glove_xgb, 
          average = 'weighted'
          )

resultsPT.head(10)

/home/mstauffer/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mstauffer/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Precision,Recall,F1 score,support
TFIDF + SVM,0.947996,0.951389,0.946878,NaN
TFIDF + XGBoost,0.927313,0.930556,0.926580,NaN
Emb FT Skip 300D + SVM,0.930072,0.937500,0.930466,NaN
Emb FT CBOW 300D + SVM,0.893617,0.895833,0.891195,NaN
Emb W2V Skip 300D + SVM,0.936474,0.937500,0.933651,NaN
Emb Glove 300D + SVM,0.912706,0.916667,0.910056,NaN
Emb FT Skip 300D + XGBoost,0.895734,0.895833,0.892506,NaN
Emb FT CBOW 300D + XGBoost,0.899561,0.902778,0.894835,NaN
Emb W2V Skip 300D + XGBoost,0.926134,0.909722,0.911945,NaN
Emb Glove 300D + XGBoost,0.903890,0.895833,0.891115,NaN


# Comentários finais

A combinação Glove 300D + SVM precisou de 40000 iterações para convergir. A combinação FastText CBOW 300D + SVM, com 40000 iterações, não convergiu.

Os resultados permitem a elaboração de algumas hipóteses:
- Para conjuntos pequenos de texto, o uso de XGBoost não necessariamente traz melhorias em relação ao baseline SVM.
- O mesmo pode ser dito do uso de embeddings: a combinação que atingiu os melhores resultados, FastText Skip-Gram 300D + SVM, ainda assim não superou o baseline TFIDF + SVM.
- O tempo de treinamento do baseline é menor do que em todas as outras combinações. Isso pesa ainda mais no trade-off: parece que para conjuntos pequenos de texto, o mais simples é o melhor.

Evidentemente, para concluir quaiquer dessas hipóteses acima, seria necessário mais testes com mais conjuntos de dados de cunho semelhante.